In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory setup
dataset_dir = os.path.expanduser('~/Downloads/mvtec-loco-ad-DatasetNinja')

train_dir = os.path.join(dataset_dir, 'train')
test_good_dir = os.path.join(dataset_dir, 'test')

# Preprocessing
img_height, img_width = 224, 224  # Resize dimensions, adjust as needed
batch_size = 32

test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)

# Since we've changed train_datagen, recreate train_generator
train_generator = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'train', 'img'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input',  # Autoencoders don't have labels, input images are also targets
    color_mode='rgb',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'test', 'img'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input',
    color_mode='rgb',
    shuffle=False)

# print(os.listdir(train_dir))



In [ ]:
import matplotlib.pyplot as plt

def display_images(image_batch, num_images=8):
    fig, axes = plt.subplots(1, num_images, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(image_batch, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Fetch a batch of images from the train generator
image_batch, _ = next(train_generator)  # Gets the next batch. Each batch has 'batch_size' images.

# Display the first 'num_images' images from this batch
display_images(image_batch[:8])  # Adjust the number to display as needed


In [ ]:
import matplotlib.pyplot as plt

def display_images(image_batch, num_images=8):
    fig, axes = plt.subplots(1, num_images, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(image_batch, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# Fetch a batch of images from the train generator
image_batch, _ = next(test_generator)  # Gets the next batch. Each batch has 'batch_size' images.

# Display the first 'num_images' images from this batch
display_images(image_batch[:8])  # Adjust the number to display as needed


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization

# Define the Sequential model
autoencoder = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(3, (3, 3), activation='sigmoid', padding='same')
])

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# You can now view the model summary
autoencoder.summary()


In [ ]:
history = autoencoder.fit(train_generator, epochs=30, validation_data=train_generator)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()


In [ ]:
def compute_anomaly_scores(model, generator, steps):
    anomaly_scores = []
    for _ in range(steps):
        original_images, _ = next(generator)
        reconstructed_images = model.predict_on_batch(original_images)
        batch_scores = tf.reduce_mean(tf.abs(original_images - reconstructed_images), axis=(1, 2, 3))
        anomaly_scores.extend(batch_scores.numpy())
    return anomaly_scores

# You'll need to specify the number of steps manually, as it depends on your dataset size and batch size.
# Calculate steps as the total number of images divided by the batch size, and round up if necessary.
steps = np.ceil(test_generator.samples / test_generator.batch_size)

# Compute anomaly scores for the test set
test_anomaly_scores = compute_anomaly_scores(autoencoder, test_generator, int(steps))


In [ ]:
def visualize_anomalies_with_scores(model, generator, threshold, num_images):
    plt.figure(figsize=(10, 2.5 * num_images))
    for i in range(num_images):
        original_images, _ = next(generator)
        reconstructed_images = model.predict_on_batch(original_images)

        # Compute anomaly scores for the current batch
        anomaly_scores = np.mean(np.abs(original_images - reconstructed_images), axis=(1, 2, 3))

        for j in range(len(original_images)):
          # Display only the first `num_images` images
          if i * len(original_images) + j >= num_images:
              break

          # Plot original image
          ax = plt.subplot(num_images, 3, 3*j+1)
          plt.imshow(original_images[j])
          plt.title(f"Original {i*len(original_images)+j+1}")
          plt.axis('off')

          # Plot reconstructed image
          ax = plt.subplot(num_images, 3, 3*j+2)
          plt.imshow(reconstructed_images[j])
          plt.title(f"Reconstructed {i*len(original_images)+j+1}")
          plt.axis('off')

          # Plot difference
          difference = np.abs(original_images[j] - reconstructed_images[j])
          ax = plt.subplot(num_images, 3, 3*j+3)
          plt.imshow(difference)
          anomaly_score = anomaly_scores[j]

          if anomaly_score > threshold:
            print("ANOMALY : \n", anomaly_score)
            title = "Anomaly"
          else:
            title =  "Normal"
            print("NORMAL : \n", anomaly_score)

          plt.title(f"{title} Score: {anomaly_score:.4f}")
          plt.axis('off')

    plt.tight_layout()
    plt.show()



In [ ]:
threshold = np.percentile(test_anomaly_scores, 0.065)
print(sorted(test_anomaly_scores))
print(threshold)
for ele in test_anomaly_scores:
  if ele > threshold:
    print("ANOMALY :", ele)
  else:
    print("NORMAL :", ele)

visualize_anomalies_with_scores(autoencoder, test_generator, threshold, num_images=30)